<a href="https://colab.research.google.com/github/oraziotorre/MomentumShiftAI/blob/main/ModelsDevelopment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf

In [1]:
# Importiamo il dataset ottenuto tramite le operazioni di Data Preprocessing
dataset = pd.read_csv("tennis_dataset.csv")

dataset

#**Train-Test split**
Per ridurre il rischio di data leakage, abbiamo progettato un processo di training-test split che include diversi accorgimenti mirati:

- Distribuzione delle tipologie di punteggi: Assicuriamo che le stesse percentuali di tipologie di punteggi presenti nel dataset originale siano mantenute sia nel training set che nel test set. Ad esempio, se nel dataset originale il 10% delle istanze ha punteggi 4-3, garantiamo che anche nel training e nel test split sia presente il 10% di istanze con punteggi 4-3. In questo modo la divisione dei dati di training e test sarà rappresentativa del dataset complessivo(SERVE???)

- Possiamo fare anche un training test split temporale perchè i dati nel dataset sono già ordinati dal più nuovo al più vecchio

- Da metà dataset in poi sono presenti le istanze invertite quindi bisogna mantenere l'istanza e l'istanza invertita nello stesso split

Questi accorgimenti ci permettono di ridurre al minimo il rischio di data leakage e migliorare la validità delle performance del modello.

#**Model 1**